# Introduction

To create a new trigger in PostgreSQL, you follow these steps:
* First, create a trigger function using the `CREATE FUNCTION` statement.
* Second, bind the trigger function to a table by using the `CREATE TRIGGER` statement.

# Create trigger function syntax

A **trigger function** is similar to a regular user-defined function. However, a trigger function does not take any arguments and has a return value of **type** `TRIGGER`.
```postgresql
CREATE FUNCTION trigger_function()
   RETURNS TRIGGER
   LANGUAGE PLPGSQL
AS $$
BEGIN
   -- trigger logic
END;
$$ 
```

> Note that you can create a trigger function using any language supported by PostgreSQL.

A trigger function receives data about its calling environment through a special structure called `TriggerData`, which contains a set of local variables. For example, `OLD` and `NEW` represent the states of the row in the table before or after the triggering event.

PostgreSQL also provides other local variables preceded by `TG_`, such as `TG_WHEN`, and `TG_TABLE_NAME`.

After creating a trigger function, you can bind it to one or more trigger events such as `INSERT`, `UPDATE`, and `DELETE`.

# CREATE TRIGGER statement

The `CREATE TRIGGER` statement allows you to create a new trigger.

```postgresql
CREATE TRIGGER trigger_name
   {BEFORE | AFTER} { event }
   ON table_name
   [FOR [EACH] { ROW | STATEMENT }]
       EXECUTE PROCEDURE trigger_function 
```
* The **event** can be `INSERT`, `DELETE`, `UPDATE`, or `TRUNCATE`.
* Define the type of triggers, which can be:
    - The **row-level trigger** that is specified by the `FOR EACH ROW` clause.
    - The **statement-level trigger** that is specified by the `FOR EACH STATEMENT` clause.
* Finally, give the name of the trigger function after the `EXECUTE PROCEDURE` keywords.


# CREATE TRIGGER example